In [2]:
import numpy

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/tmp/ipykernel_3403284/2172125874.py", line 1, in <module>
    import numpy
ModuleNotFoundError: No module named 'numpy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 89, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/IPython/core/ultratb.py", line 1457, in structured_tra

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForSeq2SeqLM.from_pretrained("meta-llama/Llama-2-7b-hf")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/tmp/ipykernel_3403284/1306639511.py", line 1, in <module>
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
ModuleNotFoundError: No module named 'transformers'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 89, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/home/stu5/s11/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-package

In [ ]:
# source: A speech by Abhraham Lincoln (Gettysburg Address, 1863).
sen_1= "The world will little note, nor long remember what we say here, but it can never forget what they did here."
# source: Wernicke's Aphasia from Aphasia Community INC.
sen_2= "You know that smoodle pinkered and that I want to get him round and take care of him like you want before."

: 

In [ ]:
def query_llama2(sentence, task):
    input_text = f"{task}: {sentence}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


: 

In [ ]:
tasks = [
    "constituency parse",
    "dependency parse",
    "CCG parse",
    "semantic roles",
    "abstract meaning representation (AMR)",
    "coreference resolution",
]

results = []

for sentence in [sen_1, sen_2]:
    for task in tasks:
        llama2_output = query_llama2(sentence, task)
        results.append({
            "Sentence": sentence,
            "Task": task,
            "LLaMA2": llama2_output,
        })
